In [17]:
import os
os.chdir(r"C:\Users\user\Python Programs\Resume Projects\End-to-End-Chicken-Disease-Classification")
print("Current directory:", os.getcwd())

Current directory: C:\Users\user\Python Programs\Resume Projects\End-to-End-Chicken-Disease-Classification


In [18]:
from dataclasses import dataclass
from pathlib import Path

In [19]:
@dataclass(frozen=True)
class TrainingConfig:
    root_dir:Path
    trianed_model_path:Path
    updated_base_model:Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation:bool
    params_image_size:list

In [20]:
from cnnProject.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from cnnProject.utils.common import readYamlFile,createDirectories
import tensorflow as tf

In [21]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH):
        self.config=readYamlFile(config_file_path)
        self.params=readYamlFile(params_file_path)
        createDirectories([self.config.artifacts_root])

    def get_training_config(self)-> TrainingConfig:
        training=self.config.training
        prepare_base_model=self.config.prepare_base_model
        params=self.params
        training_data=os.path.join(self.config.data_ingestion.unzip_dir,"Chicken-fecal-images")
        createDirectories([Path(training.root_dir)])

        training_config=TrainingConfig(
            root_dir=Path(training.root_dir),
            trianed_model_path=Path(training.trianed_model_path),
            updated_base_model=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
        )

        return training_config

In [22]:
import time
import tensorflow as tf

In [ ]:
class Training:
    def __init__(self,config:TrainingConfig):
        self.config=config
    
    def get_base_model(self):
        self.model=tf.keras.models.load_model(self.config.updated_base_model)
    
    #Imagine you're training a robot to identify fruit:
        #🥭 Train DataGenerator: You show it mangos in different lighting, angles, partially covered — so it learns to generalize.
        #✅ Validation DataGenerator: You show it clean, clear images of mangos — to check how well it learned.

    def train_valid_generator(self):
        datagenerator_kwargs=dict(rescale=1./255,validation_split=0.20)

        dataflow_kwargs=dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        self.valid_generator=valid_datagenerator.flow_from_directory(directory=self.config.training_data,
                                                                     subset="validation",
                                                                     shuffle=False,
                                                                     **dataflow_kwargs)
        if self.config.params_is_augmentation:
            train_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator=valid_datagenerator

        self.train_generator=train_datagenerator.flow_from_directory(directory=self.config.training_data,
                                                                     subset="training",
                                                                     shuffle=True,
                                                                     **dataflow_kwargs)
        
    @staticmethod
    def save_model(path:Path,model:tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch=self.train_generator.samples//self.train_generator.batch_size
        self.validation_steps=self.valid_generator.samples//self.valid_generator.batch_size

        self.model.fit(self.train_generator,
                       epochs=self.config.params_epochs,
                       steps_per_epoch=self.steps_per_epoch,
                       validation_steps=self.validation_steps,
                       validation_data=self.valid_generator)
        self.save_model(path=self.config.trianed_model_path,model=self.model)

# 🔧 1. Configuration Setup
# 🔹TrainingConfig dataclass holds all training-related parameters like:
# 		->Paths to models and data
# 		->Training hyperparameters (epochs, batch size, image size)
# 		->Whether to apply data augmentation
# 🔹ConfigurationManager class:
# 		->Reads config.yaml and params.yaml
# 		->Creates necessary directories
# 		->Combines config and params to return a TrainingConfig object

# 🧠 2. Training Class (Core Logic)
# 🔹 __init__: Takes TrainingConfig and stores it
# 🔹 get_base_model(): Loads a pre-trained model from the saved path (updated_base_model)
# 🔹 train_valid_generator(): 
# 		->Creates training and validation image generators:
# 			Uses ImageDataGenerator with rescale=1./255 and validation_split=0.2
# 		->Validation generator:
# 			No augmentation
# 			Used for evaluating model performance
# 		->Training generator:
# 			Uses augmentation (flip, rotate, zoom, etc.) if enabled
# 			Helps improve model generalization
# 🔹 train():Calculates:
# 		->steps_per_epoch = total training samples // batch size
# 		->validation_steps = total validation samples // batch size
# 		->Trains the model using .fit()
# 		->Saves the trained model to the specified path using save_model()
# 🔹 save_model(path, model): Static method to save the trained model to disk

In [24]:
try:
    config=ConfigurationManager()
    training_config=config.get_training_config()
    training=Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    raise e

[2025-06-28 23:13:24,841 : INFO : common : the yaml file form the path config\config.yaml loaded Sucessfully]
[2025-06-28 23:13:24,842 : INFO : common : the yaml file form the path params.yaml loaded Sucessfully]
[2025-06-28 23:13:24,843 : INFO : common : Directories created Sucessfully at artifacts]
[2025-06-28 23:13:24,844 : INFO : common : Directories created Sucessfully at artifacts\training]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
Epoch 1/50
19/19 [==============================] - 12s 612ms/step - loss: 13.7981 - accuracy: 0.5034 - val_loss: 26.5980 - val_accuracy: 0.3906
Epoch 2/50
19/19 [==============================] - 11s 585ms/step - loss: 9.5160 - accuracy: 0.6047 - val_loss: 1.0265 - val_accuracy: 0.8594
Epoch 3/50
19/19 [==============================] - 11s 585ms/step - loss: 3.7275 - accuracy: 0.7534 - val_loss: 1.4284 - val_accuracy: 0.7969
Epoch 4/50
19/19 [==============================] - 11s 586ms/step - loss: 0.9659 - accu